In [405]:
import pandas as pd
import numpy as np
import math as mt
import nltk.data
import codecs
import sys
import os
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from os import listdir
from os.path import isfile, join, isdir
from bs4 import BeautifulSoup

In [369]:
# Declare some constants
START_TOKEN = u"<s>"
END_TOKEN   = u"</s>"
UNK_TOKEN   = u"<unk>"

In [112]:
#docs_dict["d1"][0] = "Prag is great"
#docs_dict["d1"][1] = "Prag is good"
#doc = codecs.open(os.path.join(sys.path[0], "d30001t/APW19981016.0240"), 'r', 'utf-8')
#print ('\n-----\n'.join(tokenizer.tokenize(content)))
#for k, v in docs_dict.items():
#    for v1,v2 in v.items():
#        print(k,v1,v2)

In [172]:
def docs_to_sentences(ff_dict):
    docs_dict = defaultdict(lambda: defaultdict(str))
    # Loop through subfolders and read every file
    # and store the sentences in docs_dict
    for k, file_list in ff_dict.items():
        for file in file_list:
            doc = codecs.open(join(k, file), 'r', 'utf-8')
            content = doc.read()
            tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
            docs_dict[file] = {key:str(value) for (key,value) in enumerate(tokenizer.tokenize(content)) \
                              if value != 0}
        break    
    return docs_dict

In [173]:
def get_folders_and_filenames():
    subfolder = defaultdict(list)
    # Get the sunbfolders - each of which is a topic
    onlydirs = [d for d in listdir(os.getcwd()) if isdir(join(os.getcwd(), d))]
    try:
        onlydirs.remove(".ipynb_checkpoints")
    except:
        pass
    onlydirs

    # Now that the folders have been read,
    # Get the filenames that are under the d3xxx sub folders
    for idx,folder in enumerate(onlydirs):
        subdir = join(os.getcwd(), folder)
        onlyfiles = [f for f in listdir(subdir) if isfile(join(subdir, f))]
        subfolder[subdir] = onlyfiles 
    return subfolder

In [373]:
# Word processing functions
def canonicalize_digits(word):
    if any([c.isalpha() for c in word]): return word
    word = re.sub("\d", "DG", word)
    if word.startswith("DG"):
        word = word.replace(",", "") # remove thousands separator
    return word

def canonicalize_word(word, wordset=None, digits=True):
    word = word.lower()
    if digits:
        if (wordset != None) and (word in wordset): return word
        word = canonicalize_digits(word) # try to canonicalize numbers
    if (wordset == None) or (word in wordset):
        return word
    else:
        return UNK_TOKEN

def canonicalize_words(words, **kw):
    return [canonicalize_word(word, **kw) for word in words]

In [376]:
canonicalize_words(["abc", "123"])

['abc', 'DGDGDG']

In [400]:
def lemmatize(sentence):
    wordnet_lemmatizer = WordNetLemmatizer()

    punctuations="?:!.,;\"`>'``''</"
    header_words = ["<DOC>", 'DOC', "DOCNO", "\/DOCNO", "DOCTYPE", "<TXTTYPE", 
                    'TXTTYPE', "TXT", "<TX", "NEWS", "NEWSWIRE", "TEXT"]
    sentence_words = nltk.word_tokenize(sentence)
    
    # Canonicalization before lemmatization
    sentence_words = canonicalize_words(sentence_words)
    
    for word in sentence_words:
        if word in header_words:
            sentence_words.remove(word)
        if word in punctuations:
            sentence_words.remove(word)
            
    #sentence_words
    #print("{0:20}{1:20}".format("Word","Lemma"))
    
    ## Perform lemmatization
    words = None
    
    #for word in sentence_words:
    #    print ("{0:20}{1:20}".format(word,wordnet_lemmatizer.lemmatize(word)))
    words = [wordnet_lemmatizer.lemmatize(word) for word in sentence_words]
    return ' '.join(words)

In [401]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [402]:
def preprocess(files_and_sentences):
    preprocessed = defaultdict(lambda: defaultdict(str))
    for file_name, sent_list in files_and_sentences.items():
            
            preprocessed[file_name] = \
            {sent_id:lemmatize(cleanhtml(files_and_sentences.get(file_name).get(sent_id))) \
                                       for sent_id, sentence in sent_list.items() \
                     if lemmatize(cleanhtml(files_and_sentences.get(file_name).get(sent_id))) != ''}
     
    return preprocessed

In [406]:
#def intermediate_document_prepare(files_and_sentences):
for key in prepr_sentences.keys():
    corpus = [
     'This is the first document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?',
      ]
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus)
    print(vectorizer.get_feature_names())
#['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
#print(X.shape)
#(4, 9)
    return inter_doc
    

SyntaxError: invalid syntax (<ipython-input-406-6eee42b79d2e>, line 2)

In [435]:
[list(prepr_sentences.get(key).values()) for key in prepr_sentences.keys()][2]


["apw19981026.0220 news newswire cambodia 's two-party opposition asked the asian development bank monday to stop providing loan to the incumbent government which it call illegal",
 'negotiation to form the next government have become deadlocked and opposition party leader prince norodom ranariddh and sam rainsy are out of the country following threat of arrest from strongman hun sen. hun sen complained monday that the opposition wa trying to make it member return an international issue',
 "hun sen 's ruling party narrowly won a majority in election in july but the opposition _ claiming widespread intimidation and fraud _ ha denied hun sen the two-thirds vote in parliament required to approve the next government",
 'meanwhile it say the old government is holding power illegally',
 'ranariddh and sam rainsy renewed their international lobbying campaign against the old government monday in a letter to adb president mitsuo sato calling for the bank to stop lending money to it',
 "we respe

In [404]:
def load_data():
    ff_dict = get_folders_and_filenames()
    files_and_sentences = docs_to_sentences(ff_dict)
    return ff_dict, files_and_sentences

# Step 1: Load data
ff_dict, files_and_sentences = load_data()

# Step 2: Preprocess data to remove tags, do lemmatization & canonicalization
prepr_sentences = preprocess(files_and_sentences)

# Step 3: Vectorize documents and retrieve summarized individual documents
#inter_doc = intermediate_document_prepare(prepr_sentences)

# Step 4: Build TFIDF

# Step 5: Call Cuckoo search

#files_and_sentences
#prepr_sentences.get("APW19981016.0240") 
prepr_sentences

defaultdict(<function __main__.preprocess.<locals>.<lambda>>,
            {'APW19981016.0240': {0: 'apw19981016.0240 news newswire cambodian leader hun sen on friday rejected opposition party demand for talk outside the country accusing them of trying to internationalize the political crisis',
              1: "government and opposition party have asked king norodom sihanouk to host a summit meeting after a series of post-election negotiation between the two opposition group and hun sen 's party to form a new government failed",
              2: "opposition leader prince norodom ranariddh and sam rainsy citing hun sen 's threat to arrest opposition figure after two alleged attempt on his life said they could not negotiate freely in cambodia and called for talk at sihanouk 's residence in beijing",
              3: 'hun sen however rejected that',
              4: "i would like to make it clear that all meeting related to cambodian affair must be conducted in the kingdom of cambodia '' 

In [298]:
#from sklearn.feature_extraction.text import TfidfVectorizer
#tfidf = TfidfVectorizer().fit_transform(files_and_sentences.get('APW19981016.0240'))
files_and_sentences.get('APW19981016.0240')[0]


"<DOC>\n<DOCNO> APW19981016.0240 </DOCNO>\n<DOCTYPE> NEWS </DOCTYPE>\n<TXTTYPE> NEWSWIRE </TXTTYPE>\n<TEXT>\nCambodian leader Hun Sen on Friday rejected opposition parties' demands \nfor talks outside the country, accusing them of trying to ``internationalize'' \nthe political crisis."

In [312]:
lemmatize(cleanhtml(files_and_sentences.get('APW19981016.0240').get(2)))

Word                Lemma               
Opposition          Opposition          
leaders             leader              
Prince              Prince              
Norodom             Norodom             
Ranariddh           Ranariddh           
and                 and                 
Sam                 Sam                 
Rainsy              Rainsy              
citing              citing              
Hun                 Hun                 
Sen                 Sen                 
's                  's                  
threats             threat              
to                  to                  
arrest              arrest              
opposition          opposition          
figures             figure              
after               after               
two                 two                 
alleged             alleged             
attempts            attempt             
on                  on                  
his                 his                 
life            

In [126]:
## --------------- All subfunctions are list below ------------------
def initialize_cs_params():
    # Discovery rate of alien eggs/solutions
    n=25
    pa=0.25
    nest = []
    ## Change this if you want to get better results
    # Tolerance
    Tol=1.0e-5
    ## Simple bounds of the search domain
    # Lower bounds
    nd=15 
    Lb=-5000.0 * np.ones(nd);
    # Upper bounds
    Ub=5000.0 * np.ones(nd);
    nest = np.zeros((n,nd))
    # Random initial solutions
    for i in range(n):
        arr = np.array(Ub * np.random.rand(nd))
        nest[i,:] = arr.copy()
    return n, pa, Tol, nd, Lb, Ub, nest

In [127]:
## You can replace the following by your own functions
# A d-dimensional objective function
def fobj(u):
## d-dimensional sphere function sum_j=1^d (u_j-1)^2. 
#  with a minimum at (1,1, ...., 1); 
    #z=sum((u-1).^2);
    return sum([i**2 for i in u])

In [128]:
# Application of simple constraints
def simplebounds(s,Lb,Ub):
  # Apply the lower bound
    ns_tmp=np.array([j if i < j else i for i,j in zip(s,Lb) ])
    s = ns_tmp.copy()
  # Apply the upper bound
    ns_tmp=np.array([j if i > j else i for i,j in zip(s,Ub) ])
    s = ns_tmp.copy()
    return s

In [129]:
## Replace some nests by constructing new solutions/nests
def empty_nests(nest,Lb,Ub,pa):
    # A fraction of worse nests are discovered with a probability pa
    nrows = nest.shape[0]
    ncols = nest.shape[1]
    # Discovered or not -- a status vector
    # if pa=0.25, then atleast 73% of values will have 1
    K=np.random.rand(nrows, ncols) > pa

    # In the real world, if a cuckoo's egg is very similar to a host's eggs, then 
    # this cuckoo's egg is less likely to be discovered, thus the fitness should 
    # be related to the difference in solutions.  Therefore, it is a good idea 
    # to do a random walk in a biased way with some random step sizes.  
    ## New solution by biased/selective random walks
    #stepsize = rand * (nest(randperm(n),:) - nest(randperm(n),:));
    stepsize = np.random.rand() * (nest[np.random.permutation(nrows),:] - \
                                   nest[np.random.permutation(nrows),:]) 
    
    #new_nest = nest + stepsize.*K;
    new_nest = nest + stepsize * K
    for j in range(nrows):
        s=new_nest[j,:].copy()
        new_nest[j,:]=simplebounds(s,Lb,Ub)
    
    return new_nest


In [130]:
## Find the current best nest
#function [fmin,best,nest,fitness]=get_best_nest(nest,newnest,fitness)
def get_best_nest(nest, newnest, fitness):
# Evaluating all new solutions
    for j in range(nest.shape[0]):
        fnew=fobj(newnest[j,:])
        if fnew<=fitness[j]:
            fitness[j]=fnew.copy()
            nest[j,:]=newnest[j,:].copy()
    # Find the current best
    fmin, K = min(fitness), fitness.argmin()
    best=nest[K,:].copy()
    return fmin, best, nest, fitness

In [131]:
## Get cuckoos by ramdom walk
#function nest=get_cuckoos(nest,best,Lb,Ub)
def get_cuckoos(nest,best,Lb,Ub):
    # Levy flights
    #n=size(nest,1);
    n =nest.shape[0] 
    # Levy exponent and coefficient
    # For details, see equation (2.21), Page 16 (chapter 2) of the book
    # X. S. Yang, Nature-Inspired Metaheuristic Algorithms, 2nd Edition, Luniver Press, (2010).
    beta=3./2.;
    sigma=(mt.gamma(1+beta)* \
           np.sin(np.pi*beta/2)/(mt.gamma((1+beta)/2)*beta*2**((beta-1)/2)))**(1/beta);

    for j in range(n):
        s=nest[j,:].copy()
        # This is a simple way of implementing Levy flights
        # For standard random walks, use step=1;
        ## Levy flights by Mantegna's algorithm
        u=np.random.randn(len(s))*sigma;
        v=np.random.randn(len(s));
        step=u/abs(v)**(1/beta);

        # In the next equation, the difference factor (s-best) means that 
        # when the solution is the best solution, it remains unchanged.     
        stepsize=0.01*step*(s-best);
        # Here the factor 0.01 comes from the fact that L/100 should the typical
        # step size of walks/flights where L is the typical lenghtscale; 
        # otherwise, Levy flights may become too aggresive/efficient, 
        # which makes new solutions (even) jump out side of the design domain 
        # (and thus wasting evaluations).
        # Now the actual random walks or flights
        s=s+stepsize*np.random.randn(len(s));
        # Apply simple bounds/limits
        nest[j,:]=simplebounds(s,Lb,Ub);


    return nest

In [132]:
n, pa, Tol, nd, Lb, Ub, nest = None, None, None, None, None, None, None
n, pa, Tol, nd, Lb, Ub, nest = initialize_cs_params()

In [133]:
s = np.array([-5005, -5004, -5003, -5002, -5001, -5000, -4999, 0, 1, 2,3,4,5,5006,5007])
ns_tmp=np.array([j if i < j else i for i,j in zip(s,Lb) ])
s = ns_tmp
s

ns_tmp=np.array([j if i > j else i for i,j in zip(s,Ub) ])
s = ns_tmp
s


array([-5.000e+03, -5.000e+03, -5.000e+03, -5.000e+03, -5.000e+03,
       -5.000e+03, -4.999e+03,  0.000e+00,  1.000e+00,  2.000e+00,
        3.000e+00,  4.000e+00,  5.000e+00,  5.000e+03,  5.000e+03])

In [134]:
# Get the current best
fitness=10**10*np.ones(n);
[fmin,bestnest,nest,fitness]=get_best_nest(nest,nest,fitness);
N_iter=0;
## Starting iterations
while fmin > Tol:
    # Generate new solutions (but keep the current best)
    new_nest=get_cuckoos(nest,bestnest,Lb,Ub); 
    [fnew,best,nest,fitness]=get_best_nest(nest,new_nest,fitness);
    # Update the counter
    N_iter=N_iter+n; 
    # Discovery and randomization
    new_nest=empty_nests(nest,Lb,Ub,pa) ;
    
    # Evaluate this set of solutions
    [fnew,best,nest,fitness]=get_best_nest(nest,new_nest,fitness);
    # Update the counter again
    N_iter=N_iter+n;
    # Find the best objective so far  
    if fnew < fmin:
        fmin = fnew;
        bestnest = best;    
## End of iterations


In [135]:
bestnest

array([-6.83743357e-04, -1.43939613e-03, -1.11865412e-04, -5.46330457e-04,
        4.00154796e-04, -7.49000131e-05, -1.30190573e-04,  2.49556768e-04,
       -5.30469079e-05,  5.28816337e-04, -1.04153526e-03, -3.53595347e-04,
       -2.06531035e-03,  1.00363458e-04, -7.02444197e-04])

In [136]:
new_nest

array([[-1.50485800e-04, -1.91570215e-03, -1.16831751e-03,
        -6.95388396e-05,  1.30588675e-03,  8.93528248e-04,
        -2.33469590e-03,  7.46164743e-04,  1.05689593e-03,
         1.28500571e-03,  1.95292504e-03, -6.13346587e-04,
        -1.69359551e-04, -4.89106035e-04, -1.36615636e-03],
       [-3.35855531e-04, -2.30663042e-03,  1.50742126e-03,
         7.94614377e-05, -3.63309065e-04, -6.05935583e-04,
        -6.94920201e-04,  1.07156853e-03, -4.53120962e-04,
        -2.54821059e-04,  3.54496748e-03,  1.55067115e-03,
         5.59959972e-04,  6.36025127e-04, -7.85267682e-04],
       [ 9.03189919e-02, -2.74300157e+00,  1.90353133e+00,
         1.10657211e-01, -1.09896963e+01,  1.70172353e-01,
        -7.81927126e+01,  3.04491166e-01, -9.36024054e-01,
        -7.67596199e-01, -9.32133163e-01, -8.70654061e-03,
        -2.45763708e+00,  4.63336391e-01,  4.04401280e-01],
       [-7.45682261e-04, -1.06296870e+00,  7.35238478e-01,
         4.15979699e-02, -4.25696782e+00,  6.50378529

In [137]:
nest

array([[-1.59568069e-04, -2.06768779e-03, -8.61061297e-04,
         6.57142890e-04,  1.30588675e-03,  7.30481321e-04,
        -2.30023021e-03,  1.40149655e-04,  8.21507949e-04,
         1.28500571e-03,  2.14762106e-03,  1.27422755e-04,
        -6.84900023e-04, -1.14435901e-04, -1.36615636e-03],
       [-8.45702435e-04, -1.59500983e-03,  9.38743499e-04,
         8.75161089e-05, -3.63309065e-04, -1.01365268e-03,
        -8.44266057e-04,  8.54977839e-04, -1.50263427e-03,
        -2.54821059e-04,  2.52111883e-03,  1.62881369e-03,
        -6.93830712e-04,  6.36025127e-04, -7.85267682e-04],
       [ 9.03189919e-02, -2.74300157e+00,  1.90355253e+00,
         1.10657217e-01, -1.09890824e+01,  1.69420844e-01,
        -7.81927126e+01,  3.03674816e-01, -9.36086876e-01,
        -7.67916049e-01, -9.33076830e-01, -8.98552320e-03,
        -2.45763708e+00,  4.63250882e-01,  4.05043469e-01],
       [-7.45682261e-04, -9.47678697e-04, -1.77478165e-03,
        -1.18307875e-03,  1.38497146e-04, -9.47246164